In [8]:
from ieasyhydro_sdk.sdk import IEasyHydroSDK
from dotenv import load_dotenv

from ieasyhydro_sdk.filters import BasicDataValueFilters
import datetime as dt

import pandas as pd
import sys
import os

# Get the absolute path of the directory containing the current script
cwd = os.getcwd()

# Construct the path to the iEasyHydroForecast directory
forecast_dir = os.path.join(
    cwd, '..', 'iEasyHydroForecast')

# Add the forecast directory to the Python path
sys.path.append(forecast_dir)

# Import the modules from the forecast library
import tag_library as tl
import forecast_library as fl

env_file_path = "../config/.env_develop_kghm"
load_dotenv(env_file_path)
print("DEBUG: IEASYHYDRO_HOST: ", os.getenv("IEASYHYDRO_HOST"))

# Load sdk configuration from .env
ieh_sdk = IEasyHydroSDK()

predictor_dates = [dt.datetime(2024, 5, 13, 0, 0, 0), dt.datetime(2024, 5, 15, 12, 0, 0)]
predictor_dates = [dt.datetime(2023, 1, 13, 0, 0, 0), dt.datetime(2024, 5, 17, 12, 0, 0)]

# Define date filter
filters = BasicDataValueFilters(
    local_date_time__gte=predictor_dates[0],
    local_date_time__lt=predictor_dates[1]
)

sites = ["15102", "15212", "15215", "15216", "15149", "15214", "15171",
                 "15194", "15189", "15292", "15261", "15954", "15960",
                 "16059", "16096", "16100", "16936"]
sites = ["16139"]

for site in sites:
    #site = '15102'

    try:
        # Get data
        qdata = ieh_sdk.get_data_values_for_site(
            [site],
            'discharge_daily_average',
            filters=filters
        )
        qdata = pd.DataFrame(qdata['data_values'])
        #print("get_data_values_for_site:\n", qdata)
        #print(type(qdata))

        tdata = ieh_sdk.get_data_values_for_site(
            [site],
            'discharge_daily',
            filters=filters,
        )
        tdata = pd.DataFrame(tdata['data_values'])
        #print(tdata)

        # Get the first row from tdata in the wide format
        row = pd.DataFrame(tdata.iloc[-1]).transpose()
        #print(row)

        # add the row to qdata
        qdata = pd.concat([qdata, row])
        print(qdata)
        print(f"{site} : {round(sum(qdata['data_value']), 3)}")

    except:
        print(f"site {site} not in DB")
        continue


DEBUG: IEASYHYDRO_HOST:  http://localhost:8881
    data_value     local_date_time       utc_date_time
0     3.526478 2023-01-13 13:00:00 2023-01-13 07:00:00
1     3.526478 2023-01-14 13:00:00 2023-01-14 07:00:00
2     3.526478 2023-01-15 13:00:00 2023-01-15 07:00:00
3     3.526478 2023-01-16 13:00:00 2023-01-16 07:00:00
4     3.526478 2023-01-17 13:00:00 2023-01-17 07:00:00
..         ...                 ...                 ...
484  64.582051 2024-05-13 14:00:00 2024-05-13 08:00:00
485  69.007191 2024-05-14 14:00:00 2024-05-14 08:00:00
486  71.274745 2024-05-15 14:00:00 2024-05-15 08:00:00
487  69.007191 2024-05-16 14:00:00 2024-05-16 08:00:00
978  69.007191 2024-05-17 10:00:00 2024-05-17 04:00:00

[489 rows x 3 columns]
16139 : 49549.698


In [18]:
# Get meteo sites
meteo_sites = ieh_sdk.get_meteo_sites()
meteo_sites = pd.DataFrame(meteo_sites)
print(meteo_sites['site_code'])
print(type(meteo_sites['site_code'][0]))

0     15256
1     15283
2     15285
3     15287
4     15312
      ...  
63    38353
64    16135
65    15016
66    38358
67    15030
Name: site_code, Length: 68, dtype: object
<class 'str'>


In [23]:
# Get station meta data
station_meta = ieh_sdk.get_data_values_for_site(
    ["15212"],
    'decade_precipitation',
    filters=filters
)
print(station_meta['variable'])
station_meta_df = pd.DataFrame([station_meta['variable']])
station_meta_df

{'variable_code': '0018', 'variable_name': 'Precipitation', 'unit': 'mm/day', 'variable_type': 'decade_precipitation'}


,variable_code,variable_name,unit,variable_type
0,0018,Precipitation,mm/day,decade_precipitation


In [9]:
# Get weather station data
decade_precip = ieh_sdk.get_data_values_for_site(
    ["15212"],
    'decade_precipitation',
    filters=filters
)
decade_precip = pd.DataFrame(decade_precip['data_values'])
decade_precip

,data_value,local_date_time,utc_date_time
0,5.0,2023-01-15 13:00:00,2023-01-15 07:00:00
1,0.0,2023-01-25 13:00:00,2023-01-25 07:00:00
2,8.0,2023-02-05 13:00:00,2023-02-05 07:00:00
3,9.0,2023-02-15 13:00:00,2023-02-15 07:00:00
4,9.0,2023-03-05 13:00:00,2023-03-05 07:00:00
5,26.0,2023-03-15 13:00:00,2023-03-15 07:00:00
6,12.0,2023-03-25 13:00:00,2023-03-25 07:00:00
7,23.0,2023-04-05 14:00:00,2023-04-05 08:00:00
8,42.0,2023-04-15 14:00:00,2023-04-15 08:00:00
9,2.0,2023-04-25 14:00:00,2023-04-25 08:00:00


In [10]:
monthly_precip = ieh_sdk.get_data_values_for_site(
    ["15212"],
    'monthly_precipitation',
    filters=filters
)
monthly_precip = pd.DataFrame(monthly_precip['data_values'])
monthly_precip

,data_value,local_date_time,utc_date_time
0,8.000000,2023-01-15 13:00:00,2023-01-15 07:00:00
1,22.000000,2023-02-15 13:00:00,2023-02-15 07:00:00
2,15.666667,2023-03-15 13:00:00,2023-03-15 07:00:00
3,22.333333,2023-04-15 14:00:00,2023-04-15 08:00:00
4,996.000000,2023-05-15 14:00:00,2023-05-15 08:00:00
5,27.000000,2024-01-15 13:00:00,2024-01-15 07:00:00
6,22.000000,2024-02-15 13:00:00,2024-02-15 07:00:00
7,82.000000,2024-03-15 13:00:00,2024-03-15 07:00:00
8,106.000000,2024-04-15 14:00:00,2024-04-15 08:00:00
9,44.000000,2024-05-15 14:00:00,2024-05-15 08:00:00
